In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.simplefilter(action = "ignore")

In [3]:
LEARNING_RATE = 500
STEPS = 10000
TEST_STEPS = 100

CSV_FILE_PATH = '''./sample.csv'''
CSV_FILE_FORMAT = {
    'zipcode': str, 
    'longitude': np.float32, 
    'latitude': np.float32,
    'is_for_sale': bool,
    'property_type': str,
    'bedroom': np.float32,
    'bathroom': np.float32,
    'size': np.float32,
    'list_price': np.float32,
    'last_update': np.float32
}

# Features, Label, Column
COLUMNS = ['zipcode', 'longitude', 'latitude', 'is_for_sale', 'property_type', 'bedroom', 'bathroom', 'size', 'list_price']
CATEGORICAL_COLUMNS = ['zipcode', 'property_type']
CONTINUOUS_COLUMNS = ['bedroom', 'bathroom', 'size']
LABEL_COLUMN = 'list_price'
FEATURE_COLUMNS = ['zipcode', 'property_type', 'bedroom', 'bathroom', 'size', 'list_price']

# just show one one decimal digit
pd.options.display.float_format = '{:.1f}'.format

## Proprocess data
* load data
* randomize the index
* filter colums
* clean data

In [4]:
df = pd.read_csv(CSV_FILE_PATH, dtype=CSV_FILE_FORMAT)
df = df.reindex(np.random.permutation(df.index))
df = df[COLUMNS]

df = df[df['is_for_sale'] == True]
df = df[df['list_price'] != 0]
df = df[df['size'] != 0]
df = df.dropna()


# split data
train_df = df.sample(frac=0.9)
test_df = df.drop(train_df.index)

train_feature_label = df[FEATURE_COLUMNS]
test_features_label = test_df[FEATURE_COLUMNS]

## Prepare features

In [5]:
# put each zipcode to one bucket
zipcode = tf.contrib.layers.sparse_column_with_hash_bucket('zipcode', hash_bucket_size=1000)
property_type = tf.contrib.layers.sparse_column_with_hash_bucket('property_type', hash_bucket_size=10)
# real value
bedroom = tf.contrib.layers.real_valued_column('bedroom')
bathroom = tf.contrib.layers.real_valued_column('bathroom')
size = tf.contrib.layers.real_valued_column('size')
size_buckets = tf.contrib.layers.bucketized_column(size, boundaries=np.arange(10000, step=200).tolist())

feature_columns = [zipcode, property_type, bedroom, bathroom, size_buckets]

In [5]:
def input_fn(df):
    """
    returns (feature_columns, label)
    * feature_columns: {column_name: tf.constant} # ???
    * label: tf.constant
    """
    continous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
    categorical_cols = {k: tf.SparseTensor(
            indices = [[i, 0] for i in range(df[k].size)],
            values = df[k].values,
            shape = [df[k].size, 1])
                        for k in CATEGORICAL_COLUMNS}
    feature_columns = dict(continous_cols.items() + categorical_cols.items())
    label = tf.constant(df[LABEL_COLUMN].values)
    return feature_columns, label

def input_fn_train():
    return input_fn(train_feature_label)

In [6]:
linear_regressor = tf.contrib.learn.LinearRegressor(
    feature_columns = feature_columns,
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a5c24d0>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}


INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a5c24d0>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}


In [7]:
print "Training model..."
linear_regressor.fit(input_fn=input_fn_train, steps=STEPS)
print "Training done!"

Training model...
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


...

INFO:tensorflow:Saving checkpoints for 10000 into /var/folders/2v/75y8b1vx10l4p1mt6s086lhr0000gn/T/tmpiEgWCj/model.ckpt.


INFO:tensorflow:Saving checkpoints for 10000 into /var/folders/2v/75y8b1vx10l4p1mt6s086lhr0000gn/T/tmpiEgWCj/model.ckpt.


INFO:tensorflow:Loss for final step: 2.18808e+12.


INFO:tensorflow:Loss for final step: 2.18808e+12.


Training done!


In [8]:
print "Evaluating"
def input_fn_test():
    return input_fn(test_features_label)

print linear_regressor.evaluate(input_fn=input_fn_test, steps=TEST_STEPS)
print "Evaluation done!"

Evaluating
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


...

INFO:tensorflow:Results after 100 steps (0.001 sec/batch): loss = 1.39102e+12.


INFO:tensorflow:Results after 100 steps (0.001 sec/batch): loss = 1.39102e+12.


INFO:tensorflow:Saving evaluation summary for step 10000: loss = 1.39102e+12


INFO:tensorflow:Saving evaluation summary for step 10000: loss = 1.39102e+12


{'loss': 1.391022e+12, 'global_step': 10000}
Evaluation done!
